# DKUtils Documentation & Examples
This notebook gives a brief introduction to DKUtils and several examples using the main client that this package provides, DataKitchenClient. Multiple other clients and convenience functions & classes are provided by this package as described in the formal documention.

See the [DKUtils Documentation](https://datakitchen.github.io/DKUtils/) for a comprehensive overview of this [DKUtils Python Package](https://pypi.org/project/DKUtils/). Furthermore, the source code is publicly available on [GitHub DataKitchen/DKUtils](https://github.com/DataKitchen/DKUtils).

## Install DKUtils
Install the DKUtils package when you begin using it and anytime a newer version is available that you want to use.

In [ ]:
!pip install -U dkutils

## Imports and Global Variables
There are two ways to create a DataKitchenClient instance. The first is via its constructor by explicitly passing in a username and password. This method is typically used in General Purpose Container (GPC) python scripts:
```python
CLIENT = DataKitchenClient(username, password)
```
The second uses a DK CLI context in your home directory (see [DKCloudCommand Documentation](https://docs.datakitchen.io/articles/#!datakitchen-help/dkcloudcommand) for more information about the DK CLI).  This method is typically used for local development since it is simpler. In this example, a context is provided, but if you omit the context name, the default is used:
```python
CLIENT = create_using_context(context='Implementation')
```
In both instances, kitchen, recipe, and variation variables are optional. If any of these variables will be constant across all of your API calls, it's recommended to set them when instantiating the client:
```python
CLIENT = DataKitchenClient(username, password, kitchen=KITCHEN, recipe=RECIPE, variation=VARIATION)
CLIENT = create_using_context(context='Implementation', kitchen=KITCHEN, recipe=RECIPE, variation=VARIATION)
```


In [ ]:
import logging
import json

from datetime import datetime, timedelta
from pprint import pformat
from textwrap import indent

from dkutils.constants import INACTIVE_ORDER_STATUS_TYPES
from dkutils.datakitchen_api.datakitchen_client import DataKitchenClient, create_using_context

LOGGER = logging.getLogger()
LOGGER.addHandler(logging.StreamHandler())
LOGGER.setLevel(logging.INFO)

USERNAME = '[CHANGE_ME]'
PASSWORD = '[CHANGE_ME]'
CONTEXT  = '[CHANGE_ME]'

INDENT    = '    '
KITCHEN   = '[CHANGE_ME]'
RECIPE    = '[CHANGE_ME]'
VARIATION = '[CHANGE_ME]'

USE_CONTEXT = True
if USE_CONTEXT:
    CLIENT = create_using_context(context=CONTEXT, kitchen=KITCHEN, recipe=RECIPE, variation=VARIATION)
else:
    CLIENT = DataKitchenClient(USERNAME, PASSWORD, kitchen=KITCHEN, recipe=RECIPE, variation=VARIATION)

## [Create an Order](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.datakitchen_client.html#dkutils.datakitchen_api.datakitchen_client.DataKitchenClient.create_order)
If you didn't specify the kitchen, recipe, and variation when creating the DataKitchen Client, then you must set them before invoking `create_order`. Since we didn't set these fields above, we'll do that first here. There are two ways to do so. Since these attributes are properties with setters, one way is the following:
```python
CLIENT.kitchen = KITCHEN
CLIENT.recipe = RECIPE
CLIENT.variation = VARIATION
```
However, the client employs a somewhat [fluent interface](https://en.wikipedia.org/wiki/Fluent_interface) allowing method chaining for convenience:
```python
response = CLIENT.set_kitchen(KITCHEN).set_recipe(RECIPE).set_variation(VARIATION).create_order()
```

After setting the kitchen, recipe, and variation, simply invoke the `create_order` method. If the order takes parameters, an optional dictionary of parameters may be provided:
```python
parameters = {
    'parameter': 'value'
}
response = CLIENT.create_order(parameters=parameters)
```
If successful, the created order id is returned in the response and may be used to retrieve the order run id after it starts:
```python
order_id = response.json()['order_id']
```

In [ ]:
response = CLIENT.set_kitchen(KITCHEN).set_recipe(RECIPE).set_variation(VARIATION).create_order()
order_id = response.json()['order_id']
print(f'Created order id {order_id}')

## [Retrieve Order Runs](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.datakitchen_client.html#dkutils.datakitchen_api.datakitchen_client.DataKitchenClient.get_order_runs)
Given an order id, retrieve the associated order runs. If an order was created as shown above, only a single order run should be retrieved. The order run id is captured in the `hid` field. If the order run has not yet started, the order runs array will be empty. Therefore, you will have to wait for the order run to start. Alternatively, there is a [create_and_monitor_orders](http://dkutils.dk.io/latest/docs/apidoc/dkutils.datakitchen_api.datakitchen_client.html#dkutils.datakitchen_api.datakitchen_client.DataKitchenClient.create_and_monitor_orders) method that can invoke multiple orders at once and wait for them all to finish. Please visit the provided link for details.

In [ ]:
order_runs = CLIENT.get_order_runs(order_id)
print(f'Order runs: \n{indent(pformat(order_runs), INDENT)}')

order_run_id = order_runs[0]['hid']
print(f'Order Run ID: {order_run_id}') 

## [Retrieve Order Run Details](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.datakitchen_client.html#dkutils.datakitchen_api.datakitchen_client.DataKitchenClient.get_order_run_details)

In [ ]:
order_run_details = CLIENT.get_order_run_details(order_run_id)
print(f'Order run details: \n{indent(pformat(order_run_details), INDENT)}')

## [Retrieve Order Run Status](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.datakitchen_client.html#dkutils.datakitchen_api.datakitchen_client.DataKitchenClient.get_order_run_status)

In [ ]:
order_run_status = CLIENT.get_order_run_status(order_run_id)
print(f'Order Run Status: {order_run_status}')

## [Retrieve Kitchen Level Overrides](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.datakitchen_client.html#dkutils.datakitchen_api.datakitchen_client.DataKitchenClient.get_overrides)
Retrieve all overrides for a kitchen.

In [ ]:
overrides = CLIENT.get_overrides()
print(f'Overrides:\n{indent(pformat(overrides), INDENT)}')

## [Compare Kitchen Level Overrides](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.datakitchen_client.html#dkutils.datakitchen_api.datakitchen_client.DataKitchenClient.compare_overrides)
Compare kitchen overrides in your current kitchen with another kitchen. The `other` argument is optional - the default value is the parent kitchen.

In [ ]:
comparison = CLIENT.compare_overrides(other='IM_Production')
print(comparison)

# Kitchen Methods
A [Kitchen class](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html) provides methods for performing kitchen related API requests.

## [Create Kitchen](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.create)
Create a new kitchen and return an instance of the Kitchen class for this new kitchen.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.create_kitchen('foo', 'this is a new foo kitchen')

## [Delete Kitchen](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.delete)
Delete the current kitchen.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()
print(kitchen.delete())

## [Get Alerts](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.get_alerts)
Retrieve alerts set on this kitchen.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()
alerts = kitchen.get_alerts()
pprint(alerts)

## [Add Alerts](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.add_alerts)
Add the provided alerts to the kitchen.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()
kitchen.add_alerts({'OverDuration': ['foo@gmail.com']})

## [Delete Alerts](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.delete_alerts)
Delete the provided kitchen alerts.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()
kitchen.delete_alerts({'OverDuration': ['foo@gmail.com']})

## Get Kitchen Settings
Retrieve kitchen settings JSON.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()
kitchen_settings = kitchen._get_settings()
pprint(kitchen_settings)

## [Get Staff](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.get_staff)
Retrieve the staff and their associated roles assigned to this kitchen.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()
staff = kitchen.get_staff()
pprint(staff)

## [Delete Staff](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.delete_staff)
Delete the provided staff from this kitchen.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()
response = kitchen.delete_staff([
    'foo@datakitchen.io',
    'bar@datakitchen.io',
])
pprint(response)
staff = kitchen.get_staff()
pprint(staff)

## [Add Staff](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.add_staff)
Add the provided staff to this kitchen.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()

response = kitchen.add_staff({
    'Developer': [
        'foo@datakitchen.io',
        'bar@datakitchen.io',
    ]
})
pprint(response)
staff = kitchen.get_staff()
pprint(staff)

## [Update Staff](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.kitchen.html#dkutils.datakitchen_api.kitchen.Kitchen.update_staff)
Update roles for the provided staff to this kitchen.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
kitchen = client.get_kitchen()
response = kitchen.update_staff({
    'Admin': [
        'foo@datakitchen.io',
        'bar@datakitchen.io',
    ]
})
pprint(response)
staff = kitchen.get_staff()
pprint(staff)

# Recipe Methods
A [Recipe class](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.recipe.html#dkutils.datakitchen_api.recipe.Recipe) provides methods for performing recipe related API requests.

## [Create Recipe](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.recipe.html#dkutils.datakitchen_api.recipe.Recipe.create)
Create a new recipe in the kitchen set on the provided client and return a Recipe object

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
Recipe.create(client, '<RECIPE_NAME>', '<RECIPE_DESCRIPTION>')

## [Delete Recipe](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.recipe.html#dkutils.datakitchen_api.recipe.Recipe.delete)
Delete this recipe from the provided kitchen.

In [ ]:
recipe = Recipe(client, '<RECIPE_NAME>')
recipe.delete('<KITCHEN_NAME>')

## [Get Recipe Files](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.recipe.html#dkutils.datakitchen_api.recipe.Recipe.get_recipe_files)
Retrieve all the files for this recipe in the provided kitchen.

In [ ]:
recipe = Recipe(client, '<RECIPE_NAME>')
recipe_files = recipe.get_recipe_files('<KITCHEN_NAME>')
pprint(recipe_files)

## [Get Node Files](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.recipe.html#dkutils.datakitchen_api.recipe.Recipe.get_node_files)
Retrieve all the files associated with the provided list of nodes.

In [ ]:
recipe = Recipe(client, '<RECIPE_NAME>')
node_files = recipe.get_node_files('<KITCHEN_NAME>', ['<NODE_NAME_1>', '<NODE_NAME_2>', '<NODE_NAME_3>'])
pprint(node_files)

## [Update Recipe Files](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.recipe.html#dkutils.datakitchen_api.recipe.Recipe.update_recipe_files)
Update the files for this recipe in the provided kitchen.

In [ ]:
recipe = Recipe(client, '<RECIPE_NAME>')
updated_files = {
    'new.txt': 'Hello World\nTest New',
    'description.json': '{\n    "description": "Foo"\n}'
}
recipe.update_recipe_files('<KITCHEN_NAME>', updated_files)

## [Delete Recipe Files](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.recipe.html#dkutils.datakitchen_api.recipe.Recipe.delete_recipe_files)
Delete the provided files from this recipe in the provided kitchen.

In [ ]:
recipe = Recipe(client, '<RECIPE_NAME>')
delete_files = [
    'new.txt',
    'description.json',
]
recipe.delete_recipe_files('<KITCHEN_NAME>', delete_files)


# Vault Methods
A [Vault class](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.vault.html#module-dkutils.datakitchen_api.vault) provides methods for modifying a vault's configuration and associated secrets. To retrieve an instance of this class, call `get_vault()` on an instance of the Kitchen class. There are two types of vaults in the DataKitchen platform: global and kitchen. By default, the kitchen vault is modified by the methods below. However, most methods accommodate altering the global vault by passing an `is_global=True` keyword argument (as shown below). 

## [Get Vault Config](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.vault.html#dkutils.datakitchen_api.vault.Vault.get_config)
Return the vault configuration for the global or kitchen vault.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
vault = client.get_kitchen().get_vault()
pprint(vault.get_config())
pprint(vault.get_config(is_global=True))

## [Update Vault Config](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.vault.html#dkutils.datakitchen_api.vault.Vault.update_config)
Update the custom vault configuration for a Kitchen.

In [ ]:
VAULT_CONFIG {
    'prefix': '<PREFIX>',
    'vault_token': '<TOKEN>',
    'vault_url': '<URL>',
    'private': '<True|False>',
    'inheritable': '<True|False>',

}
client.kitchen = '<KITCHEN_NAME>'
vault = client.get_kitchen().get_vault()
pprint(vault.update_config(**VAULT_CONFIG))

## [Get Vault Secrets](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.vault.html#dkutils.datakitchen_api.vault.Vault.get_secrets)
Get a list of paths for all the secrets in the kitchen or global vault. No secret values are returned.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
vault = client.get_kitchen().get_vault()
pprint(vault.get_secrets())
pprint(vault.get_secrets(is_global=True))

## [Add/Update Vault Secret](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.vault.html#dkutils.datakitchen_api.vault.Vault.update_or_add_secret)
Update an existing secret value or add a new secret if one does not already exist.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
vault = client.get_kitchen().get_vault()
pprint(vault.update_or_add_secret('<SECRET_PATH>', '<SECRET_VALUE>'))
pprint(vault.update_or_add_secret('<SECRET_PATH>', '<SECRET_VALUE>', is_global=True))


## [Delete Vault Secret](https://datakitchen.github.io/DKUtils/apidoc/dkutils.datakitchen_api.vault.html#dkutils.datakitchen_api.vault.Vault.delete_secret)
Delete a secret from vault.

In [ ]:
client.kitchen = '<KITCHEN_NAME>'
vault = client.get_kitchen().get_vault()
pprint(vault.delete_secret('<SECRET_PATH>'))
pprint(vault.delete_secret('<SECRET_PATH>', is_global=True))